In [43]:
import pandas as pd
import featuretools as ft
import numpy as np
import os


Index = {
    (0 , 0): 0,
    (-1, 0): 1,
    (1 , 0): 2,
    (0 ,-1): 3,
    (0 , 1): 4
}

raw = pd.read_json("C:/Users/vikto/Desktop/Pacman-Pipeline/data/raw/sessions/session_12.json")

df = pd.json_normalize(raw["ticks"])

df.dropna
df.columns


Index(['enemy_momenta', 'enemy_positions_grid', 'enemy_positions_screen',
       'player_buffer', 'player_momentum', 'player_position_grid',
       'player_position_screen', 'score', 'tick'],
      dtype='object')

In [44]:
#read the tick_id and score into a seperate tick table

ticks = df["tick"].values
score = df["score"].values

tick_data = {
    "tick_id" : ticks,
    "score" : score
}

tick_table = pd.DataFrame(data=tick_data)
tick_table.head()


,tick_id,score
0,0,0
1,10,0
2,20,0
3,30,0
4,40,0


In [45]:
#read the player specific data into a seperate player table

posX = df["player_position_screen"].str[0]
posY = df["player_position_screen"].str[1]

#convert the player momentum buffer into the corresponding bufferIndex
bufferIndex = df["player_buffer"].apply(lambda x: Index[tuple(x)])

#convert the player momentum into the corresponding momentumIndex
momentumIndex = df["player_momentum"].apply(lambda x: Index[tuple(x)])


player_data = {
    "tick_id" : ticks, 
    "posX" : posX,
    "posY" : posY, 
    "player_bufferIndex" : bufferIndex, 
    "momentum_Index" : momentumIndex
}

player_table = pd.DataFrame(data=player_data) 
player_table.head()

,tick_id,posX,posY,player_bufferIndex,momentum_Index
0,0,600.0,440.0,0,0
1,10,600.0,440.0,0,0
2,20,600.0,440.0,0,0
3,30,600.0,440.0,0,0
4,40,600.0,440.0,0,0


In [ ]:
#read the enemy specific data into a seperate enemy table

#unnest the momenta and screen position arrays
df_long = df.explode(["enemy_momenta", "enemy_positions_screen"]).reset_index(drop=True)

df_long["enemy_id"] = df_long.groupby("tick").cumcount()


df_long[["posX", "posY"]] = pd.DataFrame(df_long["enemy_positions_screen"].tolist(), index=df_long.index)


df_long = df_long.drop(columns=["enemy_positions_screen"])

enemy_id = df_long["enemy_id"].values
momentumIndex = df_long["player_momentum"].apply(lambda x: Index[tuple(x)])
posX = df_long["posX"]
posY = df_long["posY"]

ticks = df_long["tick"].values

enemy_data = {
    "tick_id" : ticks, 
    "enemy_id" : enemy_id,
    "momentumIndex" : momentumIndex,
    "posX" : posX,
    "posY" : posY
}

enemy_table = pd.DataFrame(data=enemy_data)

enemy_table.head()

,tick_id,enemy_id,momentumIndex,posX,posY
0,0,0,0,840.0,120.0
1,0,1,0,120.0,760.0
2,0,2,0,1160.0,760.0
3,10,0,0,820.0,120.0
4,10,1,0,120.0,740.0
